In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pickle

from gpu import set_gpu
import copy as cp
from sklearn.utils import shuffle

import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from DL_utils import MLP, MLPbeta, CNN, eval_nn, get_train
from manual_nn import nn_pass
import session_new as session

import process_data as prd
from lda import train_lda, predict, eval_lda, eval_lda_ch

set_gpu()

Using TensorFlow backend.


Num GPUs Available:  1


In [8]:
data, params = prd.load_caps_train('traindata.mat')

In [9]:
ep = 30
n_dof = np.max(params[:,0])
    
# Train NNs
mlp = MLP(n_class=n_dof)
mlp_beta = MLPbeta(n_class=n_dof)
cnn = CNN(n_class=n_dof)

optimizer = tf.keras.optimizers.Adam()
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.CategoricalAccuracy(name='train_accuracy')

trainmlp, traincnn, y_train, x_train_mlp, x_train_cnn, x_train_lda, y_train_lda, x_train_aug, emg_scale, scaler = prd.prep_train_caps(data, params)

# Train neural networks
models = [mlp, mlp_beta, cnn]
for model in models:
    if isinstance(model,CNN):
        ds = traincnn
    else:
        ds = trainmlp
    
    train_mod = get_train()

    for epoch in range(ep):
        # Reset the metrics at the start of the next epoch
        train_loss.reset_states()
        train_accuracy.reset_states()

        for x, y in ds:
            train_mod(x, y, model, optimizer, train_loss, train_accuracy)

        if epoch == 0 or epoch == ep-1:
            print(
                f'Epoch {epoch + 1}, '
                f'Loss: {train_loss.result():.2f}, '
                f'Accuracy: {train_accuracy.result() * 100:.2f} '
            )
    del train_mod

# Train aligned LDA
y_train_aug = np.argmax(y_train, axis=1)[...,np.newaxis]

mlp_enc = mlp.get_layer(name='enc')
w_mlp, c_mlp,_, _, _ = train_lda(mlp_enc(x_train_mlp).numpy(),y_train_aug)

mlpbeta_enc = mlp_beta.get_layer(name='enc')
w_mlpbeta, c_mlpbeta,_, _, _ = train_lda(mlpbeta_enc(x_train_mlp).numpy(),y_train_aug)

cnn_enc = cnn.get_layer(name='enc')
temp = cnn_enc(x_train_cnn[:x_train_cnn.shape[0]//2,...]).numpy()
temp2 = np.vstack((temp,cnn_enc(x_train_cnn[x_train_cnn.shape[0]//2:,...]).numpy()))
w_cnn, c_cnn,_, _, _ = train_lda(temp2,y_train_aug)

# Train LDA
w,c, _, _, _ = train_lda(x_train_lda,y_train_lda)
w_aug,c_aug, _, _, _ = train_lda(x_train_aug,y_train_aug)

mlp_w = mlp.get_weights()
mlpb_w = mlp_beta.get_weights()
cnn_w = cnn.get_weights()

Epoch 1, Loss: 1.81, Accuracy: 27.22 


NameError: name 'sess' is not defined